This notebook the first level MRI analyses: setting up and examining a General Linear Model that produces response amplitudes for each stimulus class. For this, the data must be preprocessed. We will use GLMdenoise to do this, which is a MATLAB toolbox, and so the actual analysis will be run outside of this notebook (it takes a while and a lot of resources anyway, so this is for the best).

# Creating the design matrix

The first thing we need to do is create our design matrix. Our design matrix needs to be in the format time by conditions (where time is in TRs), with a 1 for condition onset. This will be exceedingly sparse, since each condition only shows up once per run (when we move this into matlab, we will make it a sparse matrix). We will have one of these design matrices per run.

In [ ]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# This file contains the button presses (which also show the TR onsets) 
# and the order the stimuli were presented in (along with their timing)
results = h5py.File('../data/raw_behavioral/2017-Aug-23_Noah_sess1.hdf5')

# This contains the information on each stimulus, allowing us to determine whether
# some stimuli are part of the same class or a separate one.
df = pd.read_csv("../data/stimuli/unshuffled_stim_description.csv", index_col=0)

Backticks from the scanner are recorded as 5s, which we hold onto, allowing us to quickly see how many TRs we recorded in each run (the last run is empty because we quit it as soon as it started):

In [ ]:
for run in range(9):
    n_TRs = sum(['5' in i[0] for i in results['run_%02d_button_presses' % run].value])
    print("Run %s: %s TRs" % (run, n_TRs))


We also have the onset times of all of our stimuli, as well as an identifying index we can use to look up its information. We know that we presented stimuli in classes, each of which contains 8 stimuli, but we can reconstruct that information by using the indices to look into the dataframe we loaded in.

Based on how we constructed our stimuli, two stimuli belong to the same class iff they have the same `w_a` and `w_r` (angular and radial frequency) values.

In [ ]:
reduced_df = df[['w_r', 'w_a', 'index']].set_index('index')
reduced_df = reduced_df.reindex(results['run_01_shuffled_indices'].value)

As can be seen by examining this reordered dataframe, our classes occur in batches of 8 (note that there are some that have `NaN`s for both values; these are the blank stimuli).

In [ ]:
reduced_df

One way to discover this programmatically is to find the smallest amount we have to jump by such that each subsequent entry has a different `w_r` and a different `w_a`.

In [ ]:
class_size = 0
break_out = False
while ~break_out:
    class_size += 1
    w_r = reduced_df['w_r'].values.copy()
    w_a = reduced_df['w_a'].values.copy()
    # we replace the NaNs with zeros for this calculation -- we want them be different than all
    # the other classes (and technically, the blank stimuli do have 0s in both w_r and w_a)
    nan_replace = 0
    w_r[np.isnan(w_r)] = nan_replace
    w_a[np.isnan(w_a)] = nan_replace
    tmp = np.abs(w_r[:-class_size:class_size] - w_r[class_size::class_size]) + np.abs(w_a[:-class_size:class_size] - w_a[class_size::class_size])
    class_changes = np.nonzero(tmp)[0]
    indices = np.array(range(len(tmp)))
    if len(class_changes) == len(indices):
        break_out = np.equal(class_changes, indices).all()
print("Each class is of size %s" % class_size)

We want to know what the class index of each stimulus is then, which we can find by dividing the index by the number of stimuli in each class:

In [ ]:
reduced_df['class_idx'] = reduced_df.index/class_size

Now we want to figure out what TR each stimulus was presented during. We have the time (in seconds) they appeared on screen recorded, so we want to add that information to our dataframe. In order to do so, we will first subtract off the time of the first TR (so we now have the "time of presentation from first TR") and throw out a bit of extra information. We also recorded when the start screen was turned off (that's the first entry) and there are a final number of blank stimuli, which we'll throw away as well

In [ ]:
timing = results['run_01_timing_data'].value
# Because we want to skip the first one and drop the last nblanks * 2 (since for each stimuli
# we have two entries: one for on, one for off). Finally, we only grab every other because we
# only want the on timing
timing = timing[1:-results['run_01_nblanks'].value*2:2]

# Now we get rid of the first TR
initial_TR_time = float(results['run_01_button_presses'].value[0][1])
timing = [float(i[2]) - initial_TR_time for i in timing]

# and add to our dataframe
reduced_df['Onset time (sec)'] = timing

Now we simply look for where there's a class transition, which happens every `class_size` (8) stimuli

In [ ]:
design_df = reduced_df[::class_size]

Now we need to convert these to to TR times. First we find the onset times of TRs, in seconds, relative to the first TR. We then create a giant matrix where each row is a different stimulus onset time and then, in each column, subtract a TR onset time (so this matrix will be `num_conditions x num_TRs`). If we then round this difference-in-time matrix and look for the 0s, we've found what TR onset is closest to the onset of the stimuli. Note that this won't make sense for a lot of entries; some of them start almost exactly halfway through a TR. But, because of how we defined our experiment, our class transitions should happen right around a TR onset (if the timings of the scanner and the stimulus computer were perfect, then they would be exactly the same; as it is they probably differ by a few milliseconds) and so this will work

In [ ]:
# 5 indicates a backtick from the scanner
TR_times = np.array([float(i[1]) for i in results['run_01_button_presses'].value if '5'==i[0]])
TR_times -= TR_times[0]

In [ ]:
stim_times = design_df['Onset time (sec)'].values
stim_times = np.expand_dims(stim_times,1)
stim_times = np.repeat(stim_times,len(TR_times),1)

In [ ]:
time_from_TR = np.round(stim_times - TR_times)
design_df['Onset time (TR)'] = np.where(time_from_TR==0)[1]

And we create our design matrix, iterate through throw our `design_df` and put a one where each class shows up in a TR.

In [ ]:
# Our blanks show up as having nan values, and we don't want to model them in our GLM, so we drop them.
design_df = design_df.dropna()
# because the values are 0-indexed
design_matrix = np.zeros((len(TR_times), design_df.class_idx.max()+1))

for i, row in design_df.iterrows():
    row = row.astype(int)
    design_matrix[row['Onset time (TR)'], row['class_idx']] = 1

To make sure things work correctly, we look at our axis sums: each class (axis 1) should show up exactly once and each TR (axis 0) should have 0 or 1 classes in it

In [ ]:
print("Each entry represents one of our %d classes:" % design_matrix.shape[1])
print(design_matrix.sum(0))

print("Each entry represents one of our %d TRs:" % design_matrix.shape[0])
print(design_matrix.sum(1))


And now we can look at our design matrix for this run!

In [ ]:
ax = plt.imshow(design_matrix, 'gray', aspect='auto',)
plt.title("Design matrix for run 1")
plt.xlabel("events")
plt.ylabel("TRs")

The function `sfp.first_level_analysis.design_matrix` does the above (without the visualizations and checks) and returns the resulting design matrix. The function `sfp.first_level_analysis.create_all_design_matrices` does this for multiple runs and saves them as `.mat` files so they can be read into matlab.

Actually running the first-level analysis requires matlab and should be run on the cluster (see `matlab/runGLM.m` and `matlab/runGLM.sbatch`), since they require Kendrick Kay's [GLMdenoise](http://kendrickkay.net/GLMdenoise/) package and use a lot of memory. After you've finished getting the results, examined the $R^2$ values to make sure they make sense, and realigned them to the subject's freesurfer anatomy (using `sfp.realign`), then you're ready for the following section, where we analyze these results.

# Analyzing the first-level results

After running our GLM analysis, we have the estimated amplitude responses of each voxel to each image class. Along with Noah Benson's anatomical template / Bayesian model, we also have each voxel's visual area and location in the visual field (in terms of eccentricity and polar angle). By combining the information contained within them, along with the dataframe describing each stimulus class, we can construct our tuning curves.

In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import seaborn as sns
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import h5py
import os
import itertools
import pyPyrTools.JBhelpers as jbh

In [ ]:
# This file contains the button presses (which also show the TR onsets) 
# and the order the stimuli were presented in (along with their timing)
behav_results = h5py.File('../data/raw_behavioral/2017-Aug-23_Noah_sess1.hdf5')

# This contains the information on each stimulus, allowing us to determine whether
# some stimuli are part of the same class or a separate one.
stim_df = pd.read_csv("../data/stimuli/unshuffled_stim_description.csv", index_col=0)

# Array full of the actual stimuli
stim = np.load('../data/stimuli/unshuffled.npy')

# for this, we just want any run, since they all contain the same classes and we don't care about their order
design_df, _, _ = sfp.first_level_analysis.create_design_df(behav_results, stim_df, 1)
design_df = design_df.reset_index(drop=True).sort("class_idx")
design_df = design_df[['w_r', 'w_a', 'class_idx']].set_index('class_idx')

stim_df = stim_df.set_index(['w_r', 'w_a'])
stim_df['class_idx'] = design_df.reset_index().set_index(['w_r', 'w_a'])['class_idx']
stim_df = stim_df.reset_index()

In [ ]:
subject_name = 'wl_subj042'
benson_path = "%s/%s/surf/{}.benson14_{}.mgz" % (os.environ['SUBJECTS_DIR'], subject_name)
results_path = "/mnt/Acadia/Projects/spatial_frequency_preferences/%s/20170823_prisma_pilot/MRI_first_level/stim_class/results/{}-{}.mgz" % subject_name
benson_path = benson_path.replace('{}', '%s')
results_path = results_path.replace('{}', '%s')

In [ ]:
df = sfp.first_level_analysis.create_GLM_result_df(design_df, benson_path, results_path)
df = sfp.first_level_analysis.round_freq_space_distance(df)

Here we see the different stimulus classes, as plotted in frequency space, colored by their superclass. These numbers are roughly log-spaced (doubling).

PROBLEMS:
- we have $w_r<0$ and we want $w_a<0$
- we go from 4 to 128 for spirals instead of 6 to 181

In [ ]:
g = sns.FacetGrid(df[df.voxel==0], hue='stimulus_superclass', size=5, aspect=1)
g.map(sns.plt.scatter, 'w_a', 'w_r')
g.add_legend()
_=g.ax.axis('equal')

In [ ]:
Rmin, Rmax = sfp.first_level_analysis.find_ecc_range_in_degrees(stim[0,:,:], 12)
print("Inside radius of stimulus annulus: %.02f" % Rmin)
print("Outside radius of stimulus annulus: %.02f" % Rmax)

For now, we only want to look at those voxels that lie within V1 (don't care if left or right hemisphere) and whose pRF centers lie between eccentricities of 2 and 8 degrees (based on above)

In [ ]:
df = df[(df.varea==1)&(df.eccen<8)&(df.eccen>2)]
df = df.sort_values('eccen')

For our analysis, we bin by eccentricity. We see that things are reasonably well distributed.

In [ ]:
def eccen_bin(ecc):
    for i in xrange(2,11):
        if ecc < i:
            return "%s-%s" % (i-1, i)

df['eccen_bin'] = df.eccen.apply(eccen_bin)

sns.countplot('eccen_bin', data=df, palette='Reds')

In [ ]:
tmp_df = df[df.stimulus_superclass=='circular']
g = sns.FacetGrid(df, hue='eccen_bin', palette='Reds', size=5,)
g.map(sns.regplot, 'rounded_freq_space_distance', 'model_ampl_est', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'rounded_freq_space_distance', 'model_ampl_est')
#g.map_dataframe(sf.utils.scatter_ci, 'freq_space_distance', 'model_ampl_est', 'model_std_error')
for ax in g.axes.flatten():
    ax.set_xscale('log', basex=2)
g.add_legend()

In [ ]:
g = sns.FacetGrid(df, hue='eccen_bin', palette='Reds', size=5, col='stimulus_superclass', col_wrap=2,
                  col_order=['circular', 'radial', 'forward spiral', 'reverse spiral', 'mixtures'])
g.map(sns.regplot, 'freq_space_distance', 'model_ampl_est', x_estimator=np.mean, fit_reg=False)
g.map_dataframe(sfp.utils.plot_mean, 'freq_space_distance', 'model_ampl_est')
#g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_distance', 'model_ampl_est', 'model_std_error')
for ax in g.axes:
    ax.set_xscale('log', basex=2)
g.add_legend()

In [ ]:
classes_of_interest = []
classes_of_interest.extend(df[(df.stimulus_superclass=='radial')&(df.base_freq==64)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='circular')&(df.base_freq==11)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='forward spiral')&(df.base_freq==16)].stimulus_class.unique())
classes_of_interest.extend(df[(df.stimulus_superclass=='reverse spiral')&(df.base_freq==16)].stimulus_class.unique())
classes_of_interest.append(df[df.stimulus_superclass=='mixtures'].stimulus_class.values[0])

stim_idxs = stim_df[stim_df.class_idx.isin(classes_of_interest)].index.values[::8]

In [ ]:
jbh.showIm([stim[i, :, :] for i in stim_idxs], ncols=4, zoom=.2)

In [ ]:
labels = sorted(df[df.stimulus_superclass=='mixtures'].freq_space_angle.unique())
labels = np.round([(i/np.pi*12) for i in labels])
labels = ['$\\frac{%s*\\pi}{12}$'%int(i) for i in labels]

tmp_df = df[df.stimulus_superclass=='mixtures']
g = sns.FacetGrid(tmp_df, hue='eccen_bin', palette='Reds', size=5)

'freq_space_angle', 'model_ampl_est', 'eccen_bin', tmp_df, size=5, palette='Reds')
_=g.ax.set_xticklabels(labels)

In [ ]:
g = sns.FacetGrid(df, hue='eccen_bin', palette='Reds', size=5, col='stimulus_superclass', col_wrap=2,
                  col_order=['circular', 'radial', 'forward spiral', 'reverse spiral', 'mixtures'])
#g.map(sns.regplot, 'freq_space_distance', 'model_ampl_est', x_estimator=np.mean, fit_reg=False)
#g.map_dataframe(sfp.utils.plot_mean, 'freq_space_distance', 'model_ampl_est')
g.map_dataframe(sfp.utils.scatter_ci, 'freq_space_distance', 'model_ampl_est', 'model_std_error')
for ax in g.axes:
    ax.set_xscale('log', basex=2)
g.add_legend()

In [ ]:
tmp_df = pd.DataFrame(df.groupby(['eccen_bin', 'w_r', 'w_a']).model_ampl_est.mean()).reset_index()

g = sns.FacetGrid(tmp_df, col='eccen_bin', col_wrap=4, size=5)
cbar_ax = g.fig.add_axes([.92, .3, .02, .4])  # <-- Create a colorbar axes
g.map(sfp.utils.scatter_heat, 'w_a', 'w_r', 'model_ampl_est', vmin=tmp_df['model_ampl_est'].min(), vmax=tmp_df['model_ampl_est'].max())
sns.plt.colorbar(cax=cbar_ax)
g.fig.subplots_adjust(right=.9)

In [ ]:

tmp_df = pd.pivot_table(df, 'model_ampl_est', 'rounded_freq_space_distance', 'eccen_bin')

In [ ]:
norm_df = tmp_df.copy()
for col in norm_df.columns:
    norm_df[col] = norm_df[col] / norm_df[col].max()

In [ ]:
sns.heatmap(tmp_df).invert_yaxis()


In [ ]:
sns.heatmap(norm_df).invert_yaxis()

In [ ]:
sns.distplot(df.R2.values)

# Double-check design matrix

In order to use this, run everything except for the `sfp.experiment.run` statement first, which will create pictures, one from each class, in the order the `design_df` thinks they are presented. Then run the `sfp.experiment.run` block on a two-monitor setup so one monitor can display the experiment while you have this notebook open in the other. This will allow you to make sure the stimuli are being ordered correctly.

You can also compare `design_df.index.values` to a picture of the design matrix to make sure things got transferred correctly there as well.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import h5py
import os
import pyPyrTools.JBhelpers as jbh

In [ ]:
# This file contains the button presses (which also show the TR onsets) 
# and the order the stimuli were presented in (along with their timing)
behav_results = h5py.File('../data/raw_behavioral/2017-Aug-23_Noah_sess1.hdf5')

# This contains the information on each stimulus, allowing us to determine whether
# some stimuli are part of the same class or a separate one.
stim_df = pd.read_csv("../data/stimuli/unshuffled_stim_description.csv", index_col=0)

# Array full of the actual stimuli
stim = np.load('../data/stimuli/unshuffled.npy')

# for this, we just want any run, since they all contain the same classes and we don't care about their order
design_df, stim_length, TR = sfp.first_level_analysis.create_design_df(behav_results, stim_df, 1, drop_blanks=False)
design_df = design_df.reset_index(drop=True).set_index("class_idx")

stim_df['class_idx'] = np.floor(stim_df['index']/8)
stim_df = stim_df.set_index('class_idx')
stim_df['Onset time (TR)'] = design_df['Onset time (TR)']
stim_df = stim_df.reset_index().sort('Onset time (TR)')

stim_df = stim_df.drop_duplicates('class_idx')
stim_idx = stim_df.index.values

In [ ]:
data = sfp.experiment.run('../data/stimuli/unshuffled.npy', '../data/stimuli/Noah_run01_idx.npy', None, screen=0)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[:10]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[10:20]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[20:30]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[30:40]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[40:50]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[50:60]], ncols=5, zoom=.2)

In [ ]:
jbh.showIm([stim[i,:,:] for i in stim_idx[60:]], ncols=5, zoom=.2)